<a href="https://colab.research.google.com/github/ChrisKantor/Deep-Learning/blob/main/Project%202/Deep_Learning_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Deep learning model that builds on the one made in HW3 for determining an infants pain level using vital signs
#This improved model uses LSTM to better understand the sequential readings



#Run these commands - NECCESARY to use the GPU
# export CUDNN_PATH=$(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))
# export LD_LIBRARY_PATH=${CUDNN_PATH}/lib

#To connect to local runtime: jupyter lab --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [1]:
#import libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import os
from matplotlib import pyplot as plt
import PIL
from tensorflow import keras

In [2]:
#Some hyperparameter setup
batch_size = 64           #set batch size to 64
epochs = 20               #set num epochs to 20

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# dir = 'Deep Learning/Project 2/'
# data_dir = "data/"


## Mount Google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data_dir = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/Project 2/data/'


import os
print(os.getcwd())

Num GPUs Available:  0
Mounted at /content/drive
/content


# Data Preprocessing

In [ ]:
#NEEDS FURTHER CLARIFICATION:
#If a csv file sequence has an invalid row, do we just remove it or is the whole sequence invalid?
#Can we mix the csv sequences together to get a fixed length input sequence? or should each seperate csv file be treated as its own sequence?
#Are we predicting if the baby is in pain during the next timestep?



#lots to do here - roughly follow this tutorial for general RNN processing: https://www.tensorflow.org/text/tutorials/text_generation

#need sequences of data to use as input. Each csv file is 1 sequence, and each of them could potentially have missing/invalid data in them. 602 total csv files
#Each csv file has 30 entries, but due to missing/invalid data, our cleaned sequences could have any number of entries
#After cleaning the csv files, our dataset will be compromised of a bunch of variable length sequences
#we will split our dataset up into training/validation sets on a sequence level

#the inputs at a specific timestep will be the 3 labels in the csv files (Heart Rate, Respiratory Rate, O2 level) and the output will be (either if the baby is in pain OR if the baby is in pain during the NEXT step)? -Need clarification here

    Heart Rate  Respiratory Rate  O2 Level  Pain
0          128                49        98     0
1          122                47        98     0
2          120                92        97     0
3          119                90        97     0
4          119                50        96     0
5          121                47        96     0
6          125                47        96     0
7          125                47        96     0
8          129                47        96     0
9          129                47        96     0
10         131                47        96     0
11         131                46        96     0
12         131                48        96     0
13         128                47        96     0
14         128                46        97     0
15         126                48        97     0
16         126                38        97     0
17         124                38        98     0
18         121                38        98     0
19         122      

In [21]:
#store a list of dataframes representing the data from the csv files we read in
#after reading in all of the csv files as dataframes, iterate through them, cleaning up the data and normalizing the inputs


#Should we normalize on a dataframe level or a dataset level?


data = []


#use the os library to iterate through all files in the current path
for (root,dirs,files) in os.walk(data_dir, topdown=True):
  #open each CSV file, we don't want to open any other type of file
  for f in files:
    if f.endswith(".csv"):
      with open(os.path.join(root, f), mode ='r') as csvFile:

        #read in the csv file
        csvData = csv.reader(csvFile)

        #store the data temporarily before we turn it into a pandas dataframe
        temp = []


        #check if each row is valid
        #one way to tell if the data is invalid is if the label is NOT [0, 1, 2]. This means there was something wrong with one of the sensors at the time of capture
        #another way is to check if the heart rate, respiratory rate, or o2 level is too low or too high
        #for heart Rate, we want a range of 40 - 200
        #for Respiratory Rate, we want a range of 15 - 90
        #for o2 level, we want a rate of 80 - 100

        for row in csvData:
          try:
            if row[4] in "012" and (int(row[1]) >= 40 and int(row[1]) <= 200) and (int(row[2]) >= 15 and int(row[2]) <= 90) and (int(row[3]) >= 80 and int(row[3]) <= 100):   #data is valid, so add it to the main csv
              #row[0] just stores the row # in the csv file, so it is not needed in the training dataset
              temp.append([row[1], row[2], row[3], row[4]])

          #Catches errors that occur when an element cannot be parsed into an integer. In this case the row is invalid so we can skip it
          except ValueError:
            continue

        #if this valid and cleaned sequence is NOT empty, add it to our dataset
        if len(temp) > 0:
          df = pd.DataFrame(temp, columns = ['Heart Rate', 'Respiratory Rate', 'O2 Level', 'Label'])
          data.append(df)

print(len(data))

487


In [22]:
#count sequence lengths and how often they appear
freqMap = {}
for d in data:
  if d.shape not in freqMap.keys():
    freqMap[d.shape] = 1

  else:
    freqMap[d.shape] += 1

for key, value in freqMap.items():
    print(f'{key} -> {value}')


#split data into training/validation sets using an 85/15 split
data = np.asarray(data, dtype="object")
np.random.shuffle(data)
trainData, valData = np.split(data, [int(0.85*len(data))])

print(len(trainData), len(valData))

#remove the labels from the trainingData and validation data, also storing them as a list
trainLabels = []
valLabels = []

for d in trainData:
  label = d.pop('Label')
  trainLabels.append(label)

for d in valLabels:
  label = d.pop('Label')
  valLabels.append(label)

(31, 4) -> 381
(30, 4) -> 31
(14, 4) -> 2
(29, 4) -> 21
(5, 4) -> 1
(11, 4) -> 1
(21, 4) -> 4
(4, 4) -> 1
(7, 4) -> 1
(27, 4) -> 10
(28, 4) -> 15
(25, 4) -> 4
(26, 4) -> 6
(17, 4) -> 2
(22, 4) -> 2
(24, 4) -> 2
(23, 4) -> 2
(15, 4) -> 1
413 74


In [ ]:
#normalize data using only the training set
#find the min/max of each column throughout all of the dataframes in the training dataset
#save the values, then iterate through the training and validation dataset and use min-max normalization
#df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())

# Model Setup

# Model Training

# Testing Script